In [341]:
#import all the essential libraries and packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
from scipy import stats
from scipy.stats import t, ttest_ind_from_stats
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline
from pandas.io.json import json_normalize


In [342]:
import datetime
from datetime import datetime, timedelta
import requests
import sys
import json
from bs4 import BeautifulSoup


def last_30_days(date):                                       # function to calculate last 30 days
    date_list = [date - timedelta(days=x) for x in range(30)]
    return sorted(date_list)

def from_date(x):                                           # function to calculate the from_param
    from_date=[]
    for i in range(len(x)):
        from_date.append(last_30_days(datetime.today())[i].date())
    return from_date

def to_date(x):                                         # function to calculate the to_param
    to_date=[]
    for i in range(len(x)):
        to_date.append(last_30_days(datetime.today())[i].date())
    return to_date
    

from_date_list = from_date(last_30_days(datetime.today()))
to_date_list = to_date(last_30_days(datetime.today()))


In [343]:
from newsapi import NewsApiClient

url = 'http://newsapi.org/v2/everything?'
newsapi_1 = NewsApiClient(api_key='cf478da0d8784f26877dc94b06b0f8e8')


data_1 = pd.DataFrame()

for (from_date,to_date) in zip(from_date_list,to_date_list):
    all_articles = newsapi_1.get_everything(q='covid OR coronavirus OR pandemic',language='en',sort_by='relevancy',page_size=100,from_param=from_date,to=to_date)
    d=json_normalize(all_articles['articles'])
    data_1 = data_1.append(d)
data_1

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Justine Calma,Tracing the link between your phone and the next pandemic,"Mining for gold, coltan, and other minerals used in electronics could inadvertently lead to another pandemic. It’s a concern scientists had before COVID-19 emerged, and the threat continues with each new generation of smartphones.",https://www.theverge.com/21314971/phone-mining-pandemic-link-coronavirus-spillover-minerals-electronics,https://cdn.vox-cdn.com/thumbor/vJFCxBp_0GA3iwiCc-OzWf4o-pI=/0x128:2040x1196/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/20069286/Container.jpg,2020-07-07T14:00:00Z,"Viruses can jump from animals to humans mining minerals that are used to make smartphones\r\nWhen humans cut their way through forests to dig up gold and other minerals that go into phones, computers, … [+5289 chars]",the-verge,The Verge
1,Tom Simonite,Covid Drives Real Businesses to Tap Deepfake Technology,Coronavirus restrictions make it harder and more expensive to shoot videos. So some companies are turning to synthetic media instead.,https://www.wired.com/story/covid-drives-real-businesses-deepfake-technology/,"https://media.wired.com/photos/5efe224130e7532f3baa1d0a/191:100/w_1280,c_limit/deepfakes-commercial-1220400443.jpg",2020-07-07T11:00:00Z,"This month, advertising giant WPP will send unusual corporate training videos to tens of thousands of employees worldwide. A presenter will speak in the recipients language and address them by name, … [+3442 chars]",wired,Wired
2,https://www.facebook.com/bbcnews,Coronavirus: Kenyan schools to remain closed until 2021,Final-year exams for primary and secondary pupils have also been cancelled because of Covid-19.,https://www.bbc.co.uk/news/world-africa-53325741,https://ichef.bbci.co.uk/news/1024/branded_news/EB57/production/_113274206_gettyimages-532537504.jpg,2020-07-07T17:55:11Z,Image copyrightAFPImage caption\r\n The education ministry said it would explore how to make online learning accessible to all students\r\nAll schools in Kenya will remain closed until next January becau… [+1560 chars],bbc-news,BBC News
3,"Yessenia Funes on Earther, shared by Brian Kahn to Gizmodo",California’s Firefighter Prison Camps go on Lockdown as Coronavirus Rages,"California’s wildland firefighting crew isn’t what it would usually be this time of year. The state depends on 2,200 incarcerated people to help work the fire line during wildfires. However, the coronavirus is ravaging through the state’s prison system. As a …",https://earther.gizmodo.com/california-s-firefighter-prison-camps-go-on-lockdown-as-1844282834,"https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/bj7dfveh04cxn9fzwud1.jpg",2020-07-07T13:00:00Z,"Californias wildland firefighting crew isnt what it would usually be this time of year. The state depends on 2,200 incarcerated people to help work the fire line during wildfires. However, the corona… [+3626 chars]",None,Gizmodo.com
4,None,Tom Hanks compares Covid-19 to WWII,"Actor Tom Hanks appeared on NBC's ""Today"" show to discuss his new movie, Greyhound, and compared wartime with the pandemic.",https://www.cnn.com/videos/entertainment/2020/07/07/tom-hanks-coronavirus-greyhound-movie-orig-jk.cnn,https://cdn.cnn.com/cnnnext/dam/assets/200103175454-tom-hanks-super-tease.jpg,2020-07-07T19:08:20Z,Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds.,cnn,CNN
...,...,...,...,...,...,...,...,...,...
95,Medha Singh,Disney's surprise profit bolsters futures ahead of economic data - Reuters,"U.S. stock futures rose on Wednesday after Disney squeezed out a quarterly profit despite taking a $5-billion charge due to the pandemic, while investors awaited data on private payrolls and the service sector to gauge the country's economic health.",https://www.reuters.com/article/us-usa-stocks-idUSKCN2511H4,https://s1.reutersmedia.net/resources/r/?m=

In [344]:
newsapi_2 = NewsApiClient(api_key='b50f5b54f58c4d6f85449b398aad75ed')

data_2 = pd.DataFrame()

for (from_date,to_date) in zip(from_date_list,to_date_list):
    all_articles = newsapi_2.get_everything(q='covid OR coronavirus OR pandemic',language='en',sort_by='publishedAt',page_size=100,from_param=from_date,to=to_date)
    d=json_normalize(all_articles['articles'])
    data_2 = data_2.append(d)
data_2

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,The Economist,Sunak’s summer statement,"A giveaway budget, vast in normal times, fails to impress",https://www.economist.com/britain/2020/07/08/sunaks-summer-statement,https://www.economist.com/sites/default/files/20200711_BRP501.jpg,2020-07-08T00:00:00Z,"ON JULY 8TH Rishi Sunak, the chancellor, delivered what the Treasury called a summer economic update. Although it was not officially a budget, which would have required new forecasts and policy costi… [+3865 chars]",None,The Economist
1,404443,AirLabs,"AirLabs is a system that provides codes for airports, airlines, cities, and aircrafts. Also, it provides information about countries, routes, and timezones. This data can be used for SEO purposes or to improve the functionality of search and meta-search servi…",https://www.programmableweb.com/api/airlabs,https://www.programmableweb.com/sites/default/files/styles/facebook_scale_width_200/public/AirLabsAPI_Image.png?itok=lNgos7Lk,2020-07-08T00:00:00Z,"Coronavirus Developer Resource Center\r\nCOVID-19 APIs, SDKs, coverage, open source code and other related dev resources »",None,Programmableweb.com
2,"Gonne, Hubert","Air travel restrictions in a pandemic: Easy to implement, hard to assess",The shutdown of passenger air travel at the height of the COVID-19 pandemic slowed the spread of the disease but caused major economic losses for the sector. This column presents a cost-benefit analysis of the global freeze of passenger air traffic. While any…,https://voxeu.org/article/air-travel-restrictions-pandemic-easy-implement-hard-assess,https://voxeu.org/sites/default/files/cover_images/article/AdobeStock_333047323.jpeg,2020-07-08T00:00:00Z,The shutdown of passenger air travel at the height of the COVID-19 pandemic slowed the spread of the disease but caused major economic losses for the sector. This column presents a cost-benefit analy… [+13079 chars],None,voxeu.org
3,None,How to reach another planet when a pandemic is hobbling yours,"As the United States, China and the United Arab Emirates head to Mars, they must all commit to sharing what they find.",https://www.nature.com/articles/d41586-020-02000-5,https://media.nature.com/lw1024/magazine-assets/d41586-020-02000-5/d41586-020-02000-5_18153588.jpg,2020-07-08T00:00:00Z,"The Hope orbiter is put through pre-launch tests at the Mohammed bin Rashid Space Centre in Dubai.Credit: MBRSC\r\nOn 15 July 1965, humanity got its first close-up look at Mars when NASAs Mariner 4 spa… [+4656 chars]",None,Nature.com
4,Olivier Pourret,Why scientists with children who have disabilities need a different career trajectory,"As lockdowns ease, Olivier Pourret hopes that academia will take on board lessons about how to redefine career success.",https://www.nature.com/articles/d41586-020-02043-8,https://media.nature.com/lw1024/magazine-assets/d41586-020-02043-8/d41586-020-02043-8_18153506.jpg,2020-07-08T00:00:00Z,Credit: Flavia Morlachetti/Getty\r\nIve been deeply disturbed during lockdown by how many people seem to see washing their hands as a new thing. I havent shaken hands with anyone for around five years.… [+2527 chars],None,Nature.com
...,...,...,...,...,...,...,...,...,...
95,Megan Rose Dickey,Uber and Lyft face new lawsuit from CA Labor Commissioner,"Uber and Lyft are both facing another lawsuit from the office of the California Labor Commissioner alleging wage theft. Filed today, the suits argue both Uber and Lyft are misclassifying their drivers as independent contractors and aim to enforce the labor pr…",http://techcrunch.com/2020/08/05/uber-and-lyft-face-new-lawsuit-from-ca-labor-commissioner/,https://techcrunch.com/wp-content/uploads/2019/09/GettyImages-1142453877.jpg?w=616,2020-08-05T23:16:21Z,"Uber and Lyft are both facing another lawsuit from the office of the California Labor Commissioner alleging wage theft. Filed today, the suits argue both Uber and Lyft are misclassifying their driver… [+3124 chars]",techc

In [355]:
newsapi_3 = NewsApiClient(api_key='94c52cf4fa594701a4df007e26d08eac')

data_3 = pd.DataFrame()

for (from_date,to_date) in zip(from_date_list,to_date_list):
    all_articles = newsapi_3.get_everything(q='covid OR coronavirus OR pandemic',language='en',sort_by='popularity',page_size=100,from_param=from_date,to=to_date)
    d=json_normalize(all_articles['articles'])
    data_3 = data_3.append(d)
data_3

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,"Beth Skwarecki on Vitals, shared by Beth Skwarecki to Lifehacker","Risk Isn't Just About You, Asshole","In the great American spirit of independence, some people have come up with an extremely shitty take on pandemic safety precautions. It goes something like: “I feel safe doing this. If you don’t, you should stay home.”Read more...",https://vitals.lifehacker.com/risk-isnt-just-about-you-asshole-1844295983,"https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/wityjhjxtfryghwg2jcf.jpg",2020-07-07T19:15:00Z,"In the great American spirit of independence, some people have come up with an extremely shitty take on pandemic safety precautions. It goes something like: I feel safe doing this. If you dont, you s… [+2204 chars]",None,Lifehacker.com
1,Imani Bashir,What You Need To Know About Getting Immunized Before You Travel,"With the pandemic having caused massive upset within the travel industry, some have envisioned a future in which you’ll need to prove you’ve had a COVID-19 immunization before you can travel. Of course, such a vaccine doesn’t exist yet, let alone any regulati…",https://lifehacker.com/heres-what-you-need-to-know-about-immunizations-needed-1844295783,"https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/bx6jdbvhfwcdxtmnkrsg.jpg",2020-07-07T20:45:00Z,"With the pandemic having caused massive upset within the travel industry, some have envisioned a future in which youll need to prove youve had a COVID-19 immunization before you can travel. Of course… [+4578 chars]",None,Lifehacker.com
2,"Kate Dore on Two Cents, shared by Kate Dore to Lifehacker",What to Do if You Can't Pay Your Taxes Next Week,"It’s official: Taxes are due next week. Back in March, the government pushed the filing deadline to July 15, and the Treasury Department recently confirmed they won’t be moving it again. If you have been putting it off, now is the time to file your tax return…",https://twocents.lifehacker.com/what-to-do-if-you-cant-pay-your-taxes-next-week-1844294481,"https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/dhsfmyh8fn5fefw4vyjr.jpg",2020-07-07T19:30:00Z,"Its official: Taxes are due next week. Back in March, the government pushed the filing deadline to July 15, and the Treasury Department recently confirmed they wont be moving it again. If you have be… [+2118 chars]",None,Lifehacker.com
3,Christine Fisher,‘Mafia: Definitive Edition’ is delayed until September due to COVID-19,"If you were hoping to play Mafia: Definitive Edition later this summer, you’ll have to wait just a little longer. Today, 2K Games announced that the re-make of the original 2002 game will arrive on September 25th, not August 28th as originally planned. Unsurp…",https://www.engadget.com/mafia-definitive-edition-delay-september-25th-165550506.html,https://o.aolcdn.com/images/dims?resize=1200%2C630&crop=1200%2C630%2C0%2C0&quality=95&image_uri=https%3A%2F%2Fs.yimg.com%2Fos%2Fcreatr-uploaded-images%2F2020-07%2F26d81660-c06f-11ea-bf86-dffa240e3f72&client=amp-blogside-v2&signature=769ea7f559000ba96de4bfb1d1577b12a1985a2b,2020-07-07T16:55:50Z,"As a consolation for the delay, 2K Games has released a 15-second teaser, and the company plans to release an extended look at Mafia: Definitive Edition’s gameplay on July 22nd. Fans will be able to … [+1007 chars]",engadget,Engadget
4,Mariella Moon,Spot and Pepper robots will perform at spectatorless baseball games in Japan,"Sports leagues and teams around the world have been filling stadiums with stuffed toys and cardboard cutouts to stand in for fans who can’t watch in person due to the coronavirus pandemic. SoftBank, however, is taking things a step further in an effort to mak…",https://www.engadget.com/spot-pepper-robot-spectatorless-baseball-games-japan-101535505.html,ht

In [356]:
frames = [data_1, data_2, data_3]

all_articles = pd.concat(frames)

all_articles

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Justine Calma,Tracing the link between your phone and the next pandemic,"Mining for gold, coltan, and other minerals used in electronics could inadvertently lead to another pandemic. It’s a concern scientists had before COVID-19 emerged, and the threat continues with each new generation of smartphones.",https://www.theverge.com/21314971/phone-mining-pandemic-link-coronavirus-spillover-minerals-electronics,https://cdn.vox-cdn.com/thumbor/vJFCxBp_0GA3iwiCc-OzWf4o-pI=/0x128:2040x1196/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/20069286/Container.jpg,2020-07-07T14:00:00Z,"Viruses can jump from animals to humans mining minerals that are used to make smartphones\r\nWhen humans cut their way through forests to dig up gold and other minerals that go into phones, computers, … [+5289 chars]",the-verge,The Verge
1,Tom Simonite,Covid Drives Real Businesses to Tap Deepfake Technology,Coronavirus restrictions make it harder and more expensive to shoot videos. So some companies are turning to synthetic media instead.,https://www.wired.com/story/covid-drives-real-businesses-deepfake-technology/,"https://media.wired.com/photos/5efe224130e7532f3baa1d0a/191:100/w_1280,c_limit/deepfakes-commercial-1220400443.jpg",2020-07-07T11:00:00Z,"This month, advertising giant WPP will send unusual corporate training videos to tens of thousands of employees worldwide. A presenter will speak in the recipients language and address them by name, … [+3442 chars]",wired,Wired
2,https://www.facebook.com/bbcnews,Coronavirus: Kenyan schools to remain closed until 2021,Final-year exams for primary and secondary pupils have also been cancelled because of Covid-19.,https://www.bbc.co.uk/news/world-africa-53325741,https://ichef.bbci.co.uk/news/1024/branded_news/EB57/production/_113274206_gettyimages-532537504.jpg,2020-07-07T17:55:11Z,Image copyrightAFPImage caption\r\n The education ministry said it would explore how to make online learning accessible to all students\r\nAll schools in Kenya will remain closed until next January becau… [+1560 chars],bbc-news,BBC News
3,"Yessenia Funes on Earther, shared by Brian Kahn to Gizmodo",California’s Firefighter Prison Camps go on Lockdown as Coronavirus Rages,"California’s wildland firefighting crew isn’t what it would usually be this time of year. The state depends on 2,200 incarcerated people to help work the fire line during wildfires. However, the coronavirus is ravaging through the state’s prison system. As a …",https://earther.gizmodo.com/california-s-firefighter-prison-camps-go-on-lockdown-as-1844282834,"https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/bj7dfveh04cxn9fzwud1.jpg",2020-07-07T13:00:00Z,"Californias wildland firefighting crew isnt what it would usually be this time of year. The state depends on 2,200 incarcerated people to help work the fire line during wildfires. However, the corona… [+3626 chars]",None,Gizmodo.com
4,None,Tom Hanks compares Covid-19 to WWII,"Actor Tom Hanks appeared on NBC's ""Today"" show to discuss his new movie, Greyhound, and compared wartime with the pandemic.",https://www.cnn.com/videos/entertainment/2020/07/07/tom-hanks-coronavirus-greyhound-movie-orig-jk.cnn,https://cdn.cnn.com/cnnnext/dam/assets/200103175454-tom-hanks-super-tease.jpg,2020-07-07T19:08:20Z,Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds.,cnn,CNN
...,...,...,...,...,...,...,...,...,...
95,Reuters Editorial,Pavlyuchenkova opts out of U.S. Open due to COVID-19 'insecurity' - Reuters Canada,Russia's Anastasia Pavlyuchenkova said on Wednesday that she will not travel to New York to take part in the U.S. Open later this month as organisers could not guarantee her health and safety amid the COVID-19 pandemic.,https://ca.reuters.com/article/sportsNews/idCAKCN25129O-OCASP,https://s3.reutersmedia.net/resources/r/?m=02&d=20200805&t=

In [359]:
all_articles.to_csv('raw_data/raw_articles.csv', index = False)

In [412]:
df_covid = pd.read_csv('raw_data/raw_articles.csv')

In [413]:
df_covid['publishedAt'] = pd.to_datetime(df_covid['publishedAt']).apply(lambda x: x.date())

In [414]:
print("Number of Duplicate Articles:")
print(df_covid.duplicated(subset='url').sum())

Number of Duplicate Articles:
1866


In [415]:
df_covid.drop_duplicates(subset='url', keep = "first", inplace = True)
df_covid.shape

(7134, 9)

In [416]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7134 entries, 0 to 8964
Data columns (total 9 columns):
author         6489 non-null object
title          7133 non-null object
description    7088 non-null object
url            7134 non-null object
urlToImage     6787 non-null object
publishedAt    7134 non-null object
content        6837 non-null object
source.id      3917 non-null object
source.name    7134 non-null object
dtypes: object(9)
memory usage: 557.3+ KB


In [417]:
df_covid.isna().sum() # for now lets keep all the Nan values, since we will combine the title, description, and content later

author         645 
title          1   
description    46  
url            0   
urlToImage     347 
publishedAt    0   
content        297 
source.id      3217
source.name    0   
dtype: int64

In [418]:
df_covid = df_covid.dropna(subset=['content', 'description'])

In [419]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6825 entries, 0 to 8964
Data columns (total 9 columns):
author         6247 non-null object
title          6824 non-null object
description    6825 non-null object
url            6825 non-null object
urlToImage     6554 non-null object
publishedAt    6825 non-null object
content        6825 non-null object
source.id      3770 non-null object
source.name    6825 non-null object
dtypes: object(9)
memory usage: 533.2+ KB


In [420]:
df_covid.isna().sum()

author         578 
title          1   
description    0   
url            0   
urlToImage     271 
publishedAt    0   
content        0   
source.id      3055
source.name    0   
dtype: int64

In [421]:
df_covid['descr_word_count']=df_covid['description'].apply(lambda x: len(x.strip().split()))  # word count in description
df_covid['content_word_count']=df_covid['content'].apply(lambda x: len(x.strip().split()))  # word count in content
df_covid['content_unique_words']=df_covid['content'].apply(lambda x:len(set(str(x).split()))) # number of unique words in content

In [422]:
df_covid

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name,descr_word_count,content_word_count,content_unique_words
0,Justine Calma,Tracing the link between your phone and the next pandemic,"Mining for gold, coltan, and other minerals used in electronics could inadvertently lead to another pandemic. It’s a concern scientists had before COVID-19 emerged, and the threat continues with each new generation of smartphones.",https://www.theverge.com/21314971/phone-mining-pandemic-link-coronavirus-spillover-minerals-electronics,https://cdn.vox-cdn.com/thumbor/vJFCxBp_0GA3iwiCc-OzWf4o-pI=/0x128:2040x1196/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/20069286/Container.jpg,2020-07-07,"Viruses can jump from animals to humans mining minerals that are used to make smartphones\r\nWhen humans cut their way through forests to dig up gold and other minerals that go into phones, computers, … [+5289 chars]",the-verge,The Verge,34,37,32
1,Tom Simonite,Covid Drives Real Businesses to Tap Deepfake Technology,Coronavirus restrictions make it harder and more expensive to shoot videos. So some companies are turning to synthetic media instead.,https://www.wired.com/story/covid-drives-real-businesses-deepfake-technology/,"https://media.wired.com/photos/5efe224130e7532f3baa1d0a/191:100/w_1280,c_limit/deepfakes-commercial-1220400443.jpg",2020-07-07,"This month, advertising giant WPP will send unusual corporate training videos to tens of thousands of employees worldwide. A presenter will speak in the recipients language and address them by name, … [+3442 chars]",wired,Wired,20,34,32
2,https://www.facebook.com/bbcnews,Coronavirus: Kenyan schools to remain closed until 2021,Final-year exams for primary and secondary pupils have also been cancelled because of Covid-19.,https://www.bbc.co.uk/news/world-africa-53325741,https://ichef.bbci.co.uk/news/1024/branded_news/EB57/production/_113274206_gettyimages-532537504.jpg,2020-07-07,Image copyrightAFPImage caption\r\n The education ministry said it would explore how to make online learning accessible to all students\r\nAll schools in Kenya will remain closed until next January becau… [+1560 chars],bbc-news,BBC News,14,32,31
3,"Yessenia Funes on Earther, shared by Brian Kahn to Gizmodo",California’s Firefighter Prison Camps go on Lockdown as Coronavirus Rages,"California’s wildland firefighting crew isn’t what it would usually be this time of year. The state depends on 2,200 incarcerated people to help work the fire line during wildfires. However, the coronavirus is ravaging through the state’s prison system. As a …",https://earther.gizmodo.com/california-s-firefighter-prison-camps-go-on-lockdown-as-1844282834,"https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/bj7dfveh04cxn9fzwud1.jpg",2020-07-07,"Californias wildland firefighting crew isnt what it would usually be this time of year. The state depends on 2,200 incarcerated people to help work the fire line during wildfires. However, the corona… [+3626 chars]",NaN,Gizmodo.com,42,34,33
4,NaN,Tom Hanks compares Covid-19 to WWII,"Actor Tom Hanks appeared on NBC's ""Today"" show to discuss his new movie, Greyhound, and compared wartime with the pandemic.",https://www.cnn.com/videos/entertainment/2020/07/07/tom-hanks-coronavirus-greyhound-movie-orig-jk.cnn,https://cdn.cnn.com/cnnnext/dam/assets/200103175454-tom-hanks-super-tease.jpg,2020-07-07,Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds.,cnn,CNN,20,16,15
...,...,...,...,...,...,...,...,...,...,...,...,...
8960,Jonathan M. Gitlin,Formula E returns today with the first of six races in nine days,The series will hold six races in Berlin beginning on August 5 at 1pm EDT.,https://arstechnica.com/cars/2020/08/formula-e-returns-today-with-the-first-of-six-races-in-nine-days/,https://cdn.arstechnica.net/wp-content/uploads/2020/08/3-EPS606_131841_FER6581-760x380.jpg,2020-08-05,"0 with 0 post